In [1]:
# Import all libraries needed for the tutorial
# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv
# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number
import functools
from spectrum import *
from pylab import plot
from scipy import signal
import numpy as np
import time
from scipy import stats
# Enable inline plotting

In [2]:
print('Python version ' + sys.version)
print('Pandas version ' + pd.__version__)
print('Matplotlib version ' + matplotlib.__version__)

Python version 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Pandas version 0.23.0
Matplotlib version 2.2.2


In [ ]:
from datetime import timedelta, date, datetime
def KSTestPerDay(cell1,cell2,kpi,ZoneDataFrame):
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    KS = pd.DataFrame()
    for day in np.array([datetime(2018,8,7)+timedelta(days=i) for i in range(1,6)]):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1))
        mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1))

        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

In [ ]:
from datetime import timedelta, date, datetime
def KSTestPerWeekDay(cell1,cell2,kpi,ZoneDataFrame):
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][['DAY_',kpi]]   
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
  
    KS = pd.DataFrame()
    for day in range(0,7):
        dayvalue = pd.DataFrame()
        dayvalue['Day'] = [day]
        mask1 = (cell1_kpi['DAY_'].dt.dayofweek==day)
        mask2 = (cell2_kpi['DAY_'].dt.dayofweek==day)
        dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
        KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

In [16]:
def KSTestPerDayAverageParallelFunc(ZoneDataFrame,cell1,cell2,kpi):
    KS = pd.DataFrame()
    if cell1 == cell2:
        KS.at[str(cell1),str(cell2)] = float(1)
    else:
        avg = KSTestPerDay(cell1,cell2,kpi,ZoneDataFrame)['pvalue'].mean()
        KS.at[cell1,cell2] = avg
    return(KS)

In [15]:
def KSTestPerWeekDayAverageParallelFunc(ZoneDataFrame,cell1,cell2,kpi):
    KS = pd.DataFrame()
    if cell1 == cell2:
        KS.at[str(cell1),str(cell2)] = float(1)
    else:
        avg = KSTestPerWeekDay(cell1,cell2,kpi,ZoneDataFrame)['pvalue'].mean()
        KS.at[cell1,cell2] = avg
    return(KS)

In [12]:
from datetime import timedelta, date, datetime
def KSTestPerWeekDayParallelFunc(cell1,cell2,kpi,day, ZoneDataFrame):
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][['DAY_',kpi]]   
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
  
    KS = pd.DataFrame()
    dayvalue = pd.DataFrame()
    dayvalue['Day'] = [day]
    mask1 = (cell1_kpi['DAY_'].dt.dayofweek==day)
    mask2 = (cell2_kpi['DAY_'].dt.dayofweek==day)
    dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
    KS = KS.append(dayvalue,ignore_index = True)
    return(KS)

In [14]:
from datetime import timedelta, date, datetime
def KSTestPerDayParallelFunc(cell1,cell2,kpi,day,ZoneDataFrame):
    cell1_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell1][['DAY_',kpi]]
    cell1_kpi['DAY_'] = pd.to_datetime(cell1_kpi['DAY_'],infer_datetime_format=True)
    
    cell2_kpi = ZoneDataFrame.loc[ZoneDataFrame['CellName']==cell2][['DAY_',kpi]]
    cell2_kpi['DAY_'] = pd.to_datetime(cell2_kpi['DAY_'],infer_datetime_format=True)
    KS = pd.DataFrame()
    dayvalue = pd.DataFrame()
    dayvalue['Day'] = [day]
    mask1 = (cell1_kpi['DAY_'] >=day) & (cell1_kpi['DAY_'] <day+timedelta(days=1))
    mask2 = (cell2_kpi['DAY_'] >=day) & (cell2_kpi['DAY_'] <day+timedelta(days=1))

    dayvalue['pvalue'] = [stats.ks_2samp(cell1_kpi.loc[mask1][kpi],cell2_kpi.loc[mask2][kpi])[1]]
    KS = KS.append(dayvalue,ignore_index = True)
    return(KS)